In [ ]:
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import numpy as np

# Generate synthetic dataset
X, y = make_regression(n_samples=60, n_features=10, noise=0.5, random_state=42)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
y_train = y_train.reshape(-1,1)
y_test = y_test.reshape(-1,1)

# Standardize the input features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
import tensorflow as tf

class Layer:
    def __init__(self, size, input_layer=False, activation = None):
        self.size = size
        self.input_layer = input_layer
        self.activation = activation
        
    def _activationFunction(self, X):
        if self.activation == 'relu':
            return tf.nn.relu(X)
        elif self.activation == 'sigmoid':
            return tf.nn.sigmoid(X)
        elif self.activation == 'softmax':
            return tf.nn.softmax(X)
        elif self.activation == 'tanh':
            return tf.nn.tanh(X)
        else:
            return X        
      
    def _construction(self, previous):
        if self.input_layer:
            return
        
        self.w=tf.Variable(tf.random.normal(shape = (previous, self.size), mean = 0, stddev = 2 / (previous + self.size)))
        self.b = tf.Variable(tf.random.normal(shape = (1, self.size), mean = 0, stddev = 2 / (previous + self.size)))
        return self.w, self.b
        
    def _feed_forward(self, X):
        if self.input_layer:
            return
        
        self.output = self._activationFunction(X @ self.w + self.b)
        return self.output
    
    def _update_w(self, grad, lr):
        self.w.assign_sub(grad * lr)
        
    def _update_b(self, grad, lr):
        self.b.assign_sub(grad * lr)

In [ ]:
class Dense_Network:
    def __init__(self, layers_list, lr = 0.000001, max_iter = 3500):
        self.ll = layers_list
        self.lr = lr
        self.max_iter = max_iter
        self.tr_vars = []
    
        for i in range(1, len(self.ll)):
            self.ll[i]._construction(self.ll[i-1].size)
    
    def feed_forward(self, X):
        X = tf.Variable(X, dtype="float32")
        inp = X
        for i in range(1, len(self.ll)):
            inp = self.ll[i]._feed_forward(inp)
        return inp
    
    def loss(self, X, y):
        y_pred = self.feed_forward(X)
        return tf.losses.mean_squared_error(y_pred, y)
    
    def fit(self, X, y):
        X = tf.Variable(X, dtype="float32")
        y = tf.Variable(y, dtype="float32")
        
        for i in range(self.max_iter):
            with tf.GradientTape(persistent=True) as tape:
                loss = self.loss(X, y)
            
            for l in range(1, len(self.ll)):
                print('loss', loss)
                print(i)
                layer_grad_w = tape.gradient(loss, self.ll[l].w)
                self.ll[l]._update_w(layer_grad_w, lr = self.lr)
                print(layer_grad_w)
                layer_grad_b = tape.gradient(loss, self.ll[l].b)
                self.ll[l]._update_b(layer_grad_b, lr = self.lr)
                
    def predict(self, X):
        return self.feed_forward(X)

In [ ]:
nn = Dense_Network([Layer(10, input_layer=True), Layer(4), Layer(3), Layer(1)])
nn.fit(X_train_scaled, y_train)

In [ ]:
print("Mean Squared Error (DenseNetwork implemented from scratch):", mean_squared_error(y_test, y_pred))